In [1]:
from simtk import unit
from openmmtools import testsystems, cache
from openmmtools.mcmc import GHMCMove, MCMCSampler, MCRotationMove, BaseIntegratorMove, IntegratorMoveError
from openmmtools.states import ThermodynamicState, SamplerState, CompoundThermodynamicState
from simtk.openmm import CompoundIntegrator
from openmmtools import alchemy
from simtk import unit
from openmmtools.utils import RestorableOpenMMObject
from openmmtools.integrators import *
import numpy as np
import copy, sys, time
import logging
from simtk import openmm
import parmed
from openmmtools.utils import SubhookedABCMeta, Timer, RestorableOpenMMObject
from openmmtools import testsystems, alchemy
import netCDF4 as nc
from blues.integrators import AlchemicalExternalLangevinIntegrator
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
np.random.RandomState(seed=3134)
logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s', level=logging.INFO, stream=sys.stdout)

ImportError: Could not import oeommtools. SideChainMove class will be unavailable.


In [2]:
# Define parameters
temperature = 300 * unit.kelvin
collision_rate = 1 / unit.picoseconds
timestep = 1.0 * unit.femtoseconds
n_steps = 10

In [17]:
# import testsystem
from openmmtools import testsystems
ala = testsystems.AlanineDipeptideExplicit()

In [32]:
# create our alchemical state
factory = alchemy.AbsoluteAlchemicalFactory(consistent_exceptions=False)
alchemical_region = alchemy.AlchemicalRegion(alchemical_atoms=range(22),annihilate_sterics=True)
alchemical_atoms = list(alchemical_region.alchemical_atoms)
alanine_alchemical_system = factory.create_alchemical_system(
    reference_system=ala.system, alchemical_regions=alchemical_region)
alchemical_state = alchemy.AlchemicalState.from_system(alanine_alchemical_system)

In [33]:
# Enslave lambda_sterics and lambda_electrostatics to a generic lambda variable.
alchemical_state.set_alchemical_variable('lambda', 0)
#Renamed in 0.17.0 to `set_function_variable` (openmm 7.3)
print(alchemical_state.lambda_electrostatics)
print(alchemical_state.lambda_sterics)

1.0
1.0


In [34]:
# Define our alchemical function and appy to state
# The functions here turn off first electrostatic and the steric interactions
# in sequence as lambda goes from 1.0 to 0.0.
f_electrostatics = '2*(lambda-0.5)*step(lambda-0.5)'
f_sterics = '2*lambda*step_hm(0.5-lambda) + step_hm(lambda-0.5)'
alchemical_state.lambda_electrostatics = alchemy.AlchemicalFunction(f_electrostatics)
alchemical_state.lambda_sterics = alchemy.AlchemicalFunction(f_sterics)

In [35]:
# Check that we can modify the lambda varible on the state
alchemical_state.set_alchemical_variable('lambda', 0.75)
#print(dir(alchemical_state))
print(alchemical_state.lambda_electrostatics)
print(alchemical_state.lambda_sterics)

0.5
1.0


In [36]:
alchemical_state.set_alchemical_variable('lambda', 1.0)
print(alchemical_state.lambda_electrostatics)
print(alchemical_state.lambda_sterics)

1.0
1.0


In [37]:
# Create our custom State objects
# Need two different Thermodynamic State objects
# Context cache will grab correct thermodynamic state
# Keeping them in sync is in SamplerState.apply to context
# Have apply return accumulated work

thermo_state = ThermodynamicState(system=alanine_alchemical_system, temperature=300 * unit.kelvin)
compound_state = CompoundThermodynamicState(thermo_state, composable_states=[alchemical_state])

sampler_state = SamplerState(positions=ala.positions)

In [38]:
compound_state.set_alchemical_variable('lambda', 0.5)
print(compound_state.lambda_electrostatics)
print(compound_state.lambda_sterics)

0.0
1.0


In [39]:
compound_state.set_alchemical_variable('lambda', 1.0)
print(compound_state.lambda_electrostatics)
print(compound_state.lambda_sterics)

1.0
1.0


# Below try simulation outside the classes

In [40]:
# # Set Integrator
ncmc_integrator = AlchemicalNonequilibriumLangevinIntegrator(
    splitting='H R V O V R H',
    #splitting='O { V R H R V } O',
    temperature=temperature,
    collision_rate=collision_rate,
    timestep=timestep,
    nsteps_neq=n_steps,
    measure_heat=True)

# ncmc_integrator = AlchemicalExternalLangevinIntegrator(nstepsNC=n_steps,
#             alchemical_functions={
#                 'lambda_sterics': f_sterics,
#                 'lambda_electrostatics': f_electrostatics
#             },
#             splitting="H V R O R V H",
#             temperature=temperature,
#             dt=timestep,
#             nprop=1,
#             propLambda=0.3)
print(ncmc_integrator)

<openmmtools.integrators.AlchemicalNonequilibriumLangevinIntegrator; proxy of <Swig Object of type 'OpenMM::CustomIntegrator *' at 0x151b3fdc90> >


In [41]:
integrator = LangevinIntegrator(
    temperature=temperature, timestep=timestep, collision_rate=collision_rate, measure_heat=True)

compound_integrator = CompoundIntegrator()
compound_integrator.addIntegrator(ncmc_integrator)
compound_integrator.addIntegrator(integrator)
compound_integrator.setCurrentIntegrator(0)

In [42]:
sampler_state.positions

Quantity(value=array([[1.59081745, 1.19692554, 1.60887376],
       [1.53772539, 1.28994663, 1.58864236],
       [1.44705674, 1.29396154, 1.64900842],
       ...,
       [1.5180571 , 1.5578555 , 0.2357921 ],
       [1.535579  , 1.4892329 , 0.1714013 ],
       [1.4482728 , 1.610326  , 0.1965588 ]]), unit=nanometer)

In [43]:
# Grab the context and set state
context_cache = cache.global_context_cache
context, compound_integrator = context_cache.get_context(compound_state, compound_integrator)

# If we reassign velocities, we can ignore the ones in sampler_state.
sampler_state.apply_to_context(context)
context.setVelocitiesToTemperature(compound_state.temperature)

In [46]:
compound_integrator.step(1)
context.setParameter('lambda_sterics', 0.2)
print(context.getState(getEnergy=True).getPotentialEnergy())
context.setParameter('lambda_sterics', 0.4)
print(context.getState(getEnergy=True).getPotentialEnergy())

-24666.198940691276 kJ/mol
-24675.024737075903 kJ/mol


In [50]:
# Do half of the steps 
rot_step = int(n_steps/2)


for n in range(n_steps):
    compound_integrator.step(1)
    try:
        curr_integrator = compound_integrator.getIntegrator(compound_integrator.getCurrentIntegrator())
        step = curr_integrator.getGlobalVariableByName('step')
        alch_lambda = curr_integrator.getGlobalVariableByName('lambda')
        compound_state.set_alchemical_variable('lambda', alch_lambda)
        
        compound_state.apply_to_context(context)
        lambda_sterics = context.getParameter('lambda_sterics')
        Eold = curr_integrator.getGlobalVariableByName('Eold')
        print(Eold)
        Enew = curr_integrator.getGlobalVariableByName('Enew')
        print(Enew)
        
        lambda_step = curr_integrator.getGlobalVariableByName('lambda_step')
        protocol_work = curr_integrator.getGlobalVariableByName('protocol_work')
        
        print('Step:', step, 'Work:', protocol_work, 'Lambda: ', alch_lambda, 'Lambda Step: ', lambda_step, 'Sterics:', lambda_sterics, 'Elec:', compound_state.lambda_electrostatics)
        if n == rot_step-1:
            break
    except Exception as e:
        print(e)

#print(dir(compound_state))


-24359.299490499747
-24359.299490499747
Step: 1.0 Work: 0.0 Lambda:  0.1 Lambda Step:  2.0 Sterics: 0.2 Elec: -0.0
-24499.464565691276
-24499.464565691276
Step: 2.0 Work: 0.0 Lambda:  0.2 Lambda Step:  4.0 Sterics: 0.4 Elec: -0.0
-24514.774737075903
-24514.774737075903
Step: 3.0 Work: 0.0 Lambda:  0.3 Lambda Step:  6.0 Sterics: 0.6 Elec: -0.0
-24535.74123998717
-24535.74123998717
Step: 4.0 Work: 0.0 Lambda:  0.4 Lambda Step:  8.0 Sterics: 0.8 Elec: -0.0
-24559.020324452606
-24559.020324452606
Step: 5.0 Work: 0.0 Lambda:  0.5 Lambda Step:  10.0 Sterics: 1.0 Elec: 0.0


In [17]:
# APPLY ROTATION HERE

In [47]:
# Run our integrator to the end
rot_step = int(n_steps/2)
for n in range(n_steps):
    compound_integrator.step(1)
    try:
        curr_integrator = compound_integrator.getIntegrator(compound_integrator.getCurrentIntegrator())
        alch_lambda = curr_integrator.getGlobalVariableByName('lambda')
        compound_state.set_alchemical_variable('lambda', alch_lambda)
        step = curr_integrator.getGlobalVariableByName('step')
        lambda_step = curr_integrator.getGlobalVariableByName('lambda_step')
        protocol_work = curr_integrator.getGlobalVariableByName('protocol_work')
        print('Step:', step, 'Work:', protocol_work, 'Lambda: ', alch_lambda, 'Lambda Step: ', lambda_step, 'Sterics:', compound_state.lambda_sterics, 'Elec:', compound_state.lambda_electrostatics)
        if n == rot_step-1:
            break
    except Exception as e:
        print(e)

Step: 8.0 Work: 0.0 Lambda:  0.8 Lambda Step:  16.0 Sterics: 1.0 Elec: 0.6000000000000001
Step: 9.0 Work: 0.0 Lambda:  0.9 Lambda Step:  18.0 Sterics: 1.0 Elec: 0.8
Step: 10.0 Work: 0.0 Lambda:  1.0 Lambda Step:  20.0 Sterics: 1.0 Elec: 1.0
Step: 10.0 Work: 0.0 Lambda:  1.0 Lambda Step:  20.0 Sterics: 1.0 Elec: 1.0
Step: 10.0 Work: 0.0 Lambda:  1.0 Lambda Step:  20.0 Sterics: 1.0 Elec: 1.0


In [18]:
# We get also velocities here even if we don't need them because we
# will recycle this State to update the sampler state object. This
# way we won't need a second call to Context.getState().
context_state = context.getState(
    getPositions=True, getVelocities=True, getEnergy=True, enforcePeriodicBox=compound_state.is_periodic)

# This is an optimization around the fact that Collective Variables are not a part of the State,
# but are a part of the Context. We do this call twice to minimize duplicating information fetched from
# the State.
# Update everything but the collective variables from the State object
sampler_state.update_from_context(context_state, ignore_collective_variables=True)
# Update only the collective variables from the Context
sampler_state.update_from_context(
    context, ignore_positions=True, ignore_velocities=True, ignore_collective_variables=False)

In [19]:
sampler_state.positions

Quantity(value=array([[ 0.19481841,  0.10117554,  0.00121941],
       [ 0.19860132,  0.21010968,  0.00142812],
       [ 0.155908  ,  0.24750274,  0.09448746],
       [ 0.14124119,  0.25556201, -0.07934875],
       [ 0.34059298,  0.26081327, -0.00204767],
       [ 0.43367076,  0.1849142 , -0.00169651],
       [ 0.3634775 ,  0.39076504, -0.00318154],
       [ 0.282085  ,  0.45041218,  0.00111682],
       [ 0.48881745,  0.46392432,  0.00548376],
       [ 0.53569311,  0.43532035,  0.09964047],
       [ 0.56711704,  0.42145589, -0.11958296],
       [ 0.50980324,  0.45038456, -0.20766963],
       [ 0.66358221,  0.47113124, -0.12996715],
       [ 0.59295863,  0.31569564, -0.12487275],
       [ 0.47099653,  0.61516815,  0.00479275],
       [ 0.35693577,  0.66410482, -0.00560002],
       [ 0.58453548,  0.68693143, -0.00144625],
       [ 0.66966236,  0.63268834, -0.0049247 ],
       [ 0.58468324,  0.83298361, -0.00097397],
       [ 0.48738745,  0.88021767,  0.01257102],
       [ 0.64512676,  0.8

## Reset the steps here so that lambda still increments

In [48]:
# Reset the variables
curr_integrator.setGlobalVariableByName('step', 0)
curr_integrator.setGlobalVariableByName('lambda_step', 0)
curr_integrator.setGlobalVariableByName('protocol_work', 0)

In [49]:
# Reconstruct a new context/integrator to repeat iteration
context, compound_integrator1 = context_cache.get_context(compound_state)
sampler_state.apply_to_context(context)
context.setVelocitiesToTemperature(compound_state.temperature)

In [22]:
# Run our integrator to the end
rot_step = int(n_steps/2)
for n in range(n_steps):
    compound_integrator.step(1)
    try:
        curr_integrator = compound_integrator.getIntegrator(compound_integrator.getCurrentIntegrator())
        alch_lambda = curr_integrator.getGlobalVariableByName('lambda')
        compound_state.set_alchemical_variable('lambda', alch_lambda)
        
        step = curr_integrator.getGlobalVariableByName('step')
        lambda_step = curr_integrator.getGlobalVariableByName('lambda_step')
        protocol_work = curr_integrator.getGlobalVariableByName('protocol_work')
        print('Step:', step, 'Work:', protocol_work, 'Lambda: ', alch_lambda, 'Lambda Step: ', lambda_step, 'Sterics:', compound_state.lambda_sterics, 'Elec:', compound_state.lambda_electrostatics)
        if n == rot_step-1:
            break
    except Exception as e:
        print(e)

Step: 1.0 Work: 0.0 Lambda:  0.1 Lambda Step:  2.0 Sterics: 0.2 Elec: -0.0
Step: 2.0 Work: 0.0 Lambda:  0.2 Lambda Step:  4.0 Sterics: 0.4 Elec: -0.0
Step: 3.0 Work: 0.0 Lambda:  0.3 Lambda Step:  6.0 Sterics: 0.6 Elec: -0.0
Step: 4.0 Work: 0.0 Lambda:  0.4 Lambda Step:  8.0 Sterics: 0.8 Elec: -0.0
Step: 5.0 Work: 0.0 Lambda:  0.5 Lambda Step:  10.0 Sterics: 1.0 Elec: 0.0


In [23]:
# Run our integrator to the end
rot_step = int(n_steps/2)
for n in range(n_steps):
    compound_integrator.step(1)
    try:
        curr_integrator = compound_integrator.getIntegrator(compound_integrator.getCurrentIntegrator())
        alch_lambda = curr_integrator.getGlobalVariableByName('lambda')
        compound_state.set_alchemical_variable('lambda', alch_lambda)

        step = curr_integrator.getGlobalVariableByName('step')
        lambda_step = curr_integrator.getGlobalVariableByName('lambda_step')
        protocol_work = curr_integrator.getGlobalVariableByName('protocol_work')
        print('Step:', step, 'Work:', protocol_work, 'Lambda: ', alch_lambda, 'Lambda Step: ', lambda_step, 'Sterics:', compound_state.lambda_sterics, 'Elec:', compound_state.lambda_electrostatics)
        if n == rot_step-1:
            break
    except Exception as e:
        print(e)

Step: 6.0 Work: 0.0 Lambda:  0.6 Lambda Step:  12.0 Sterics: 1.0 Elec: 0.19999999999999996
Step: 7.0 Work: 0.0 Lambda:  0.7 Lambda Step:  14.0 Sterics: 1.0 Elec: 0.3999999999999999
Step: 8.0 Work: 0.0 Lambda:  0.8 Lambda Step:  16.0 Sterics: 1.0 Elec: 0.6000000000000001
Step: 9.0 Work: 0.0 Lambda:  0.9 Lambda Step:  18.0 Sterics: 1.0 Elec: 0.8
Step: 10.0 Work: 0.0 Lambda:  1.0 Lambda Step:  20.0 Sterics: 1.0 Elec: 1.0


# Below try it inside classes

In [24]:
class NCMCMove(BaseIntegratorMove):

    def __init__(self,
                 timestep=1.0 * unit.femtosecond,
                 collision_rate=10.0 / unit.picoseconds,
                 n_steps=1000,
                 temperature=300 * unit.kelvin,
                 reassign_velocities=True,
                 **kwargs):
        super(NCMCMove, self).__init__(n_steps=n_steps, **kwargs)
        self.timestep = timestep
        self.temperature = temperature
        self.collision_rate = collision_rate
        self.n_accepted = 0  # Number of accepted steps.
        self.n_proposed = 0  # Number of attempted steps.

    @property
    def fraction_accepted(self):
        """Ratio between accepted over attempted moves (read-only).

        If the number of attempted steps is 0, this is numpy.NaN.

        """
        if self.n_proposed == 0:
            return np.NaN
        # TODO drop the casting when stop Python2 support
        return float(self.n_accepted) / self.n_proposed

    @property
    def statistics(self):
        """The acceptance statistics as a dictionary."""
        return dict(n_accepted=self.n_accepted, n_proposed=self.n_proposed)

    @statistics.setter
    def statistics(self, value):
        self.n_accepted = value['n_accepted']
        self.n_proposed = value['n_proposed']

    def reset_statistics(self):
        """Reset the internal statistics of number of accepted and attempted moves."""
        self.n_accepted = 0
        self.n_proposed = 0

    def apply(self, thermodynamic_state, sampler_state, integrator_idx):
        """Propagate the state through the integrator.

            This updates the SamplerState after the integration. It also logs
            benchmarking information through the utils.Timer class.

            Parameters
            ----------
            thermodynamic_state : openmmtools.states.ThermodynamicState
               The thermodynamic state to use to propagate dynamics.
            sampler_state : openmmtools.states.SamplerState
               The sampler state to apply the move to. This is modified.

            See Also
            --------
            openmmtools.utils.Timer

            """
        move_name = self.__class__.__name__  # shortcut
        timer = Timer()

        # Check if we have to use the global cache.
        if self.context_cache is None:
            context_cache = cache.global_context_cache
        else:
            print('Found context cache')
            context_cache = self.context_cache

        # Create integrator.
        integrator = self._get_integrator(thermodynamic_state)
       

        # Create context.
        timer.start("{}: Context request".format(move_name))
        context, integrator = context_cache.get_context(thermodynamic_state, integrator)
        timer.stop("{}: Context request".format(move_name))
        logger.debug("{}: Context obtained, platform is {}".format(move_name, context.getPlatform().getName()))

        #print('SamplerContext compat:', sampler_state.is_context_compatible(context))
        self.integrator_idx = integrator_idx
        integrator.setCurrentIntegrator(self.integrator_idx)
        #integrator = integrator.getIntegrator(integrator_idx)
        #RestorableOpenMMObject.restore_interface(integrator)
        #integrator.pretty_print()
        #print('Current Integrator:', integrator)

        # Perform the integration.
        for attempt_counter in range(self.n_restart_attempts + 1):

            #If we reassign velocities, we can ignore the ones in sampler_state.
            sampler_state.apply_to_context(context, ignore_velocities=self.reassign_velocities)
            if self.reassign_velocities:
                context.setVelocitiesToTemperature(thermodynamic_state.temperature)

            # Subclasses may implement _before_integration().
            self._before_integration(context, thermodynamic_state)

            #specify nc integrator variables to report in verbose output
            try:
                # Run dynamics.
                timer.start("{}: step({})".format(move_name, self.n_steps))
                #integrator.step(self.n_steps)
                
                ###NML:
                for n in range(self.n_steps):
                    integrator.step(1)
                    try:
                        curr_integrator = integrator.getIntegrator(integrator.getCurrentIntegrator())
                        step = curr_integrator.getGlobalVariableByName('step')
                        alch_lambda = curr_integrator.getGlobalVariableByName('lambda')
                        thermodynamic_state.set_alchemical_variable('lambda', alch_lambda)

                        lambda_step = curr_integrator.getGlobalVariableByName('lambda_step')
                        protocol_work = curr_integrator.getGlobalVariableByName('protocol_work')

                        print('Step:', step, 'Work:', protocol_work, 'Lambda: ', alch_lambda, 'Lambda Step: ', lambda_step, 'Sterics:', compound_state.lambda_sterics, 'Elec:', compound_state.lambda_electrostatics)
                    except Exception as e:
                        print(e)

                
                
                
            except Exception as e:
                print(e)
                # Catches particle positions becoming nan during integration.
                restart = True
            else:
                timer.stop("{}: step({})".format(move_name, self.n_steps))

                # We get also velocities here even if we don't need them because we
                # will recycle this State to update the sampler state object. This
                # way we won't need a second call to Context.getState().
                context_state = context.getState(
                    getPositions=True,
                    getVelocities=True,
                    getEnergy=True,
                    enforcePeriodicBox=thermodynamic_state.is_periodic)

                # Check for NaNs in energies.
                potential_energy = context_state.getPotentialEnergy()
                print('potential_energy', potential_energy)
                restart = np.isnan(potential_energy.value_in_unit(potential_energy.unit))

            # Restart the move if we found NaNs.
            if restart:
                err_msg = ('Potential energy is NaN after {} attempts of integration '
                           'with move {}'.format(attempt_counter, self.__class__.__name__))

                # If we are on our last chance before crash, try to re-initialize context
                if attempt_counter == self.n_restart_attempts - 1:
                    logger.error(err_msg + ' Trying to reinitialize Context as a last-resort restart attempt...')
                    context.reinitialize()
                    sampler_state.apply_to_context(context)
                    thermodynamic_state.apply_to_context(context)
                # If we have hit the number of restart attempts, raise an exception.
                elif attempt_counter == self.n_restart_attempts:
                    # Restore the context to the state right before the integration.
                    sampler_state.apply_to_context(context)
                    logger.error(err_msg)
                    raise IntegratorMoveError(err_msg, self, context)
                else:
                    logger.warning(err_msg + ' Attempting a restart...')
            else:
                break

        # Subclasses can read here info from the context to update internal statistics.
        self._after_integration(context, thermodynamic_state)

        # Updated sampler state.
        timer.start("{}: update sampler state".format(move_name))
        # This is an optimization around the fact that Collective Variables are not a part of the State,
        # but are a part of the Context. We do this call twice to minimize duplicating information fetched from
        # the State.
        # Update everything but the collective variables from the State object
        sampler_state.update_from_context(context_state, ignore_collective_variables=True)
        # Update only the collective variables from the Context
        sampler_state.update_from_context(
            context, ignore_positions=True, ignore_velocities=True, ignore_collective_variables=False)
        timer.stop("{}: update sampler state".format(move_name))

        #timer.report_timing()
        return context

    def __getstate__(self):
        serialization = super(NCMCMove, self).__getstate__()
        serialization['timestep'] = self.timestep
        serialization['collision_rate'] = self.collision_rate
        serialization.update(self.statistics)
        return serialization

    def __setstate__(self, serialization):
        super(NCMCMove, self).__setstate__(serialization)
        self.timestep = serialization['timestep']
        self.collision_rate = serialization['collision_rate']
        self.statistics = serialization

    def _get_integrator(self, thermodynamic_state):
        """Implement BaseIntegratorMove._get_integrator()."""
        # Store lastly generated integrator to collect statistics.
        ncmc_integrator = AlchemicalNonequilibriumLangevinIntegrator(
            splitting='H V R O R V H',
            #splitting='O { V R H R V } O',
            temperature=self.temperature,
            collision_rate=self.collision_rate,
            timestep=self.timestep,
            nsteps_neq=self.n_steps,
            measure_heat=True)

        integrator = LangevinIntegrator(
            temperature=self.temperature,
            timestep=self.timestep,
            collision_rate=self.collision_rate,
            measure_heat=True)
        
        compound_integrator = openmm.CompoundIntegrator()
        compound_integrator.addIntegrator(ncmc_integrator)
        compound_integrator.addIntegrator(integrator)
        return compound_integrator

    def _after_integration(self, context, thermodynamic_state):
        """Implement BaseIntegratorMove._after_integration()."""
        #Reset
        integrator = context.getIntegrator()
        curr_integrator = integrator.getIntegrator(integrator.getCurrentIntegrator())
        curr_integrator.setGlobalVariableByName('step', 0)
        curr_integrator.setGlobalVariableByName('lambda_step', 0)
        curr_integrator.setGlobalVariableByName('protocol_work', 0)
#         # Accumulate acceptance statistics.
#         ncmc_global_variables = {
#             integrator.getGlobalVariableName(index): index for index in range(integrator.getNumGlobalVariables())
#         }
#         print(ncmc_global_variables)
#         n_accepted = integrator.getGlobalVariable(ncmc_global_variables['naccept'])
#         n_proposed = integrator.getGlobalVariable(ncmc_global_variables['ntrials'])
#         self.n_accepted += n_accepted
#         self.n_proposed += n_proposed


In [25]:
class NCMCSampler(object):


    def __init__(self, thermodynamic_state=None, sampler_state=None, move=None, platform=None, ncfile=None, pdbfile=None, topology=None):
        """
        Create an MCMC sampler.
        Parameters
        ----------
        thermodynamic_state : ThermodynamicState
            The thermodynamic state to simulate
        sampler_state : SamplerState
            The initial sampler state to simulate from.
        platform : simtk.openmm.Platform, optional, default=None
            If specified, this platform will be used
        ncfile : netCDF4.Dataset, optional, default=None
            NetCDF storage file.
        """
        if thermodynamic_state is None:
            raise Exception("'thermodynamic_state' must be specified")
        if sampler_state is None:
            raise Exception("'sampler_state' must be specified")

        self.thermodynamic_state = thermodynamic_state
        self.sampler_state = sampler_state
        self.move = move
        
        # Initialize
        self.iteration = 0
        # For GHMC / Langevin integrator
        self.collision_rate = 1.0 / unit.picoseconds
        self.timestep = 2.0 * unit.femtoseconds
        self.nsteps = 500  # number of steps per update
        self.verbose = True
        self.platform = platform

        # For writing PDB files
        self.pdbfile = pdbfile
        self.topology = topology

        self._timing = dict()
        self._initializeNetCDF(ncfile)

    def _initializeNetCDF(self, ncfile):
        self.ncfile = nc.Dataset(ncfile, 'w', format='NETCDF3_64BIT')
        if self.ncfile == None:
            return

        natoms = self.thermodynamic_state.system.getNumParticles()
        self.ncfile.createDimension('iterations', None)
        self.ncfile.createDimension('atoms', natoms)  # TODO: What do we do if dimension can change?
        self.ncfile.createDimension('spatial', 3)

        self.ncfile.createVariable(
            'coordinates', 'f4', dimensions=('iterations', 'atoms', 'spatial'), zlib=True, chunksizes=(1, natoms, 3))
        self.ncfile.createVariable(
            'box_vectors', 'f4', dimensions=('iterations', 'spatial', 'spatial'), zlib=True, chunksizes=(1, 3, 3))
        self.ncfile.createVariable('potential', 'f8', dimensions=('iterations',), chunksizes=(1,))
        self.ncfile.createVariable('sample_positions_time', 'f4', dimensions=('iterations',), chunksizes=(1,))
    
    def update(self, integrator_idx=0):
        """
        Update the sampler with one step of sampling.
        """
        if self.verbose:
            print("." * 80)
            print("MCMC sampler iteration %d" % self.iteration)
            
        initial_time = time.time()
        
        # Take some steps
        self.context = self.move.apply(self.thermodynamic_state, self.sampler_state, integrator_idx)
        
        # Get new sampler state.
        self.sampler_state.update_from_context(self.context)
        

        final_time = time.time()
        elapsed_time = final_time - initial_time
        self._timing['sample positions'] = elapsed_time 
        
        if self.verbose:
            final_energy = self.context.getState(getEnergy=True).getPotentialEnergy() * self.thermodynamic_state.beta
            print('Final energy is %12.3f kT' % (final_energy))
            print('elapsed time %8.3f s' % elapsed_time)

        if self.ncfile:
            self.ncfile.variables['coordinates'][
                self.iteration, :, :] = self.sampler_state.positions[:, :] / unit.nanometers
            for k in range(3):
                self.ncfile.variables['box_vectors'][self.iteration,
                                                     k, :] = self.sampler_state.box_vectors[k, :] / unit.nanometers
            self.ncfile.variables['potential'][self.iteration] = self.thermodynamic_state.beta * self.context.getState(
                getEnergy=True).getPotentialEnergy()
            self.ncfile.variables['sample_positions_time'][self.iteration] = elapsed_time

        # Increment iteration count
        self.iteration += 1

        if self.verbose:
            print("." * 80)
        if self.pdbfile is not None:
            print("Writing frame...")
            from simtk.openmm.app import PDBFile
            with open(self.pdbfile, 'w') as pdbfile:
                PDBFile.writeFile(self.topology, self.sampler_state.positions, pdbfile, self.iteration)
            #PDBFile.writeModel(self.topology, self.sampler_state.positions, self.pdbfile, self.iteration)
                pdbfile.flush()

    def run(self, n_iterations=1, integrator_idx=0):
        """
        Run the sampler for the specified number of iterations
        Parameters
        ----------
        niterations : int, optional, default=1
            Number of iterations to run the sampler for.
        """
        for iteration in range(n_iterations):
            self.update(integrator_idx)
                        
    def minimize(self,
                 tolerance=1.0 * unit.kilocalories_per_mole / unit.angstroms,
                 max_iterations=100,
                 context_cache=None):
        """Minimize the current configuration.

        Parameters
        ----------
        tolerance : simtk.unit.Quantity, optional
            Tolerance to use for minimization termination criterion (units of
            energy/(mole*distance), default is 1*kilocalories_per_mole/angstroms).
        max_iterations : int, optional
            Maximum number of iterations to use for minimization. If 0, the minimization
            will continue until convergence (default is 100).
        context_cache : openmmtools.cache.ContextCache, optional
            The ContextCache to use for Context creation. If None, the global cache
            openmmtools.cache.global_context_cache is used (default is None).

        """
        if context_cache is None:
            context_cache = cache.global_context_cache

        timer = Timer()

        # Use LocalEnergyMinimizer
        timer.start("Context request")
        integrator = openmm.VerletIntegrator(1.0 * unit.femtosecond)
        context, integrator = context_cache.get_context(self.thermodynamic_state, integrator)
        self.sampler_state.apply_to_context(context)
        logger.debug("LocalEnergyMinimizer: platform is %s" % context.getPlatform().getName())
        logger.debug("Minimizing with tolerance %s and %d max. iterations." % (tolerance, max_iterations))
        timer.stop("Context request")

        timer.start("LocalEnergyMinimizer minimize")
        openmm.LocalEnergyMinimizer.minimize(context, tolerance, max_iterations)
        timer.stop("LocalEnergyMinimizer minimize")
        context_state = context.getState(
            getPositions=True,
            getVelocities=True,
            getEnergy=True,
            enforcePeriodicBox=self.thermodynamic_state.is_periodic)

        potential_energy = context_state.getPotentialEnergy()
        print(potential_energy)
        # Retrieve data.
        self.sampler_state.update_from_context(context)

        #timer.report_timing()

In [27]:
ncmc_move = NCMCMove(timestep=1.0 * unit.femtosecond, n_steps=10)
sampler = NCMCSampler(compound_state, sampler_state, move=ncmc_move, platform=None, pdbfile='test.pdb', ncfile='test.nc', topology=ala.topology)
sampler.minimize(max_iterations=0)
sampler.run(n_iterations=1, integrator_idx=0)
np.allclose(sampler.sampler_state.positions, ala.positions)

# In[44]:

sampler.run(n_iterations=1, integrator_idx=0)

2018-10-31 09:50:58,400 | DEBUG : LocalEnergyMinimizer: platform is OpenCL
2018-10-31 09:50:58,401 | DEBUG : Minimizing with tolerance 1.0 kcal/(A mol) and 0 max. iterations.
-116.744384765625 kJ/mol
................................................................................
MCMC sampler iteration 0
2018-10-31 09:50:58,681 | DEBUG : NCMCMove: Context obtained, platform is OpenCL
Step: 1.0 Work: 0.0 Lambda:  0.1 Lambda Step:  2.0 Sterics: 0.2 Elec: -0.0
Step: 2.0 Work: 0.0 Lambda:  0.2 Lambda Step:  4.0 Sterics: 0.4 Elec: -0.0
Step: 3.0 Work: 0.0 Lambda:  0.3 Lambda Step:  6.0 Sterics: 0.6 Elec: -0.0
Step: 4.0 Work: 0.0 Lambda:  0.4 Lambda Step:  8.0 Sterics: 0.8 Elec: -0.0
Step: 5.0 Work: 0.0 Lambda:  0.5 Lambda Step:  10.0 Sterics: 1.0 Elec: 0.0
Step: 6.0 Work: 0.0 Lambda:  0.6 Lambda Step:  12.0 Sterics: 1.0 Elec: 0.19999999999999996
Step: 7.0 Work: 0.0 Lambda:  0.7 Lambda Step:  14.0 Sterics: 1.0 Elec: 0.3999999999999999
Step: 8.0 Work: 0.0 Lambda:  0.8 Lambda Step:  16.0 Steri